# 1. Entorno

In [1]:
#Creamos un Notebook en Python sobre JupyterLab

# 2. Inicio de Sesión sobre SageMaker

In [1]:
#Utilitario para construir modelos de Machine Learning y Deep Learning
import sagemaker

#Iniciamos sesión en el servicio de SageMaker
sesion = sagemaker.Session()

#Obtenemos la ejecución en donde estamos trabajando
region = sesion.boto_region_name

#Verificamos
print(region)

#Obtenemos el rol de ejecución de SageMaker
#El rol de ejecución permite a SageMaker usar otros servicios de AWS
rol = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
us-east-1


# 3. Lectura de archivos

In [2]:
#Utilitario para leer archivos de datos
from sagemaker.inputs import TrainingInput

#Bucket en donde se encuentran los archivos
#IMPORTANTE: REEMPLAZAR "XXX" POR TUS INICIALES
bucket = "datasetsbdajac"

#Lectura de datos de entrenamiento
dataTrain = TrainingInput(
    f"s3://{bucket}/data/insurance_dataset_validacion_cruzada/vc2/train/", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

#Lectura de datos de validación
dataTest = TrainingInput(
    f"s3://{bucket}/data/insurance_dataset_validacion_cruzada/vc2/test/", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

# 4. Configuración del Algoritmo

In [3]:
#Importamos el utilitario para definir el entrenador del algoritmo
from sagemaker.estimator import Estimator

#Definimos el entrenador del algoritmo
entrenador = Estimator(
    image_uri = sagemaker.image_uris.retrieve("linear-learner", region), #Descargamos la implementación del algoritmo desde la región donde trabajamos
    role = rol, #Rol que ejecuta servicios sobre AWS
    instance_count = 1, #Cantidad de servidores de entrenamiento
    instance_type = "ml.m5.large", #Tipo de servidor de entrenamiento
    predictor_type = "regressor", #Tipo de predicción del algoritmo
    sagemaker_session = sesion, #Sesión de SageMaker
    base_job_name = "entrenamiento-prediccion-numerica-vc2" #Nombre del job de entrenamiento
)

#Configuramos los parametros del algoritmo
entrenador.set_hyperparameters(
    feature_dim = 11, #Cantidad de features
    predictor_type = "regressor", #Indicamos que tipo de predicción es
    normalize_data = "true", #Normalizamos los features
    normalize_label = "true" #Normalizamos el label
)

# 5. Entrenamiento del Modelo

In [4]:
#Entrenamos y validamos el modelo
#MIENTRAS SE ENTRENA EL MODELO: En SageMaker, en la sección "Jobs", en la opción "Training" podemos ver cómo el modelo se entrena
#TIEMPO DE ENTRENAMIENTO: 5 MINUTOS
entrenador.fit({"train": dataTrain, "validation": dataTest})

INFO:sagemaker:Creating training-job with name: entrenamiento-prediccion-numerica-vc2-2024-09-27-14-17-55-946


2024-09-27 14:17:57 Starting - Starting the training job...
2024-09-27 14:18:13 Starting - Preparing the instances for training...
2024-09-27 14:18:45 Downloading - Downloading input data...
2024-09-27 14:19:25 Downloading - Downloading the training image.........
2024-09-27 14:20:57 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[09/27/2024 14:21:01 INFO 139706614634304] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss'

# 6. Estadísticas del Modelo

In [5]:
#Librería para manipular los servicios de AWS
import boto3

#Nos conectamos al servicio de SageMaker
sagemakerCliente = boto3.client("sagemaker")

#Obtenemos el nombre del último entrenamiento realizado
nombreDeEntrenamiento = entrenador.latest_training_job.name

#Verificamos
print(nombreDeEntrenamiento)

#Obtenemos la descripción del entrenamiento
descripcionDeEntrenamiento = sagemakerCliente.describe_training_job(TrainingJobName = nombreDeEntrenamiento)

#Vemos toda la descripción de entrenamiento
descripcionDeEntrenamiento

#Dentro de la sub-variable "FinalMetricDataList" tenemos las métricas del modelo
#El error cuadrático medio se encuentra en "validation:mse"
#El r2 (error "porcentual") se encuentra en "validation:r2"
#IMPORTANTE: En SageMaker, en la sección "Jobs", en la opción "Training" podemos ver las estadísticas del modelo
descripcionDeEntrenamiento["FinalMetricDataList"]

entrenamiento-prediccion-numerica-vc2-2024-09-27-14-17-55-946


[{'MetricName': 'train:progress',
  'Value': 100.0,
  'Timestamp': datetime.datetime(2024, 9, 27, 14, 21, 6, tzinfo=tzlocal())},
 {'MetricName': 'validation:mae',
  'Value': 5034.4267578125,
  'Timestamp': datetime.datetime(2024, 9, 27, 14, 21, 6, tzinfo=tzlocal())},
 {'MetricName': 'train:objective_loss',
  'Value': 0.7860754132270813,
  'Timestamp': datetime.datetime(2024, 9, 27, 14, 21, 6, tzinfo=tzlocal())},
 {'MetricName': 'validation:objective_loss',
  'Value': 113637216.0,
  'Timestamp': datetime.datetime(2024, 9, 27, 14, 21, 6, tzinfo=tzlocal())},
 {'MetricName': 'validation:objective_loss:final',
  'Value': 44533536.0,
  'Timestamp': datetime.datetime(2024, 9, 27, 14, 21, 6, tzinfo=tzlocal())},
 {'MetricName': 'validation:rmse',
  'Value': 6673.34521484375,
  'Timestamp': datetime.datetime(2024, 9, 27, 14, 21, 6, tzinfo=tzlocal())},
 {'MetricName': 'validation:mse',
  'Value': 44533536.0,
  'Timestamp': datetime.datetime(2024, 9, 27, 14, 21, 6, tzinfo=tzlocal())},
 {'MetricNam

In [6]:
#Imprimimos sólo la métrica de nuestro interés
for metrica in descripcionDeEntrenamiento["FinalMetricDataList"]:
    if metrica["MetricName"] == "validation:mse":
        print(metrica["Value"])

44533536.0
